<a href="https://colab.research.google.com/github/FlanagG21/DCC-Capstone/blob/main/Genius_API_Connection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: mount to drive

from google.colab import drive
drive.mount('/content/drive')
root = '/content/drive/MyDrive/DCC Project/'


Mounted at /content/drive


In [ ]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

movies_df = pd.read_parquet(os.path.join(root, 'movies_embeddings.parquet'), engine='pyarrow')
songs_df = pd.read_parquet(os.path.join(root, 'song_embeddings.parquet'), engine='pyarrow')

In [ ]:
movies_df.head()

,id,title,overview,genres,embedding
0,27205,Inception,Cobb a skilled thief who commits corporate esp...,"Action, Science Fiction, Adventure","[0.01589059643447399, 0.11273891478776932, -0...."
1,157336,Interstellar,The adventures of a group of explorers who mak...,"Adventure, Drama, Science Fiction","[0.037922028452157974, -0.005655079614371061, ..."
2,155,The Dark Knight,Batman raises the stakes in his war on crime W...,"Drama, Action, Crime, Thriller","[0.011266704648733139, 0.032755907624959946, -..."
3,19995,Avatar,In the 22nd century a paraplegic Marine is dis...,"Action, Adventure, Fantasy, Science Fiction","[0.01744804158806801, 0.03436880186200142, 0.0..."
4,24428,The Avengers,When an unexpected enemy emerges and threatens...,"Science Fiction, Action, Adventure","[0.027801260352134705, -0.019952325150370598, ..."


In [ ]:
songs_df.head()

,title,tag,lyrics,embedding
0,Poor Poor Pitiful Me,country,Well I lay my head on the railroad track\nWait...,"[0.029062896966934204, 0.08223594725131989, -0..."
1,Cuckoos Nest,country,There is a thorn bush\nIn Outcolia\nThere is a...,"[0.0009067401406355202, -0.09515126794576645, ..."
2,Wedding Bells,country,I have the invitation that your sent me\nYou w...,"[0.04617173597216606, 0.013956493698060513, -0..."
3,Could Have Fooled Me,country,Im fading like the taillights\nOf a car that y...,"[0.015817370265722275, -0.0025993576273322105,..."
4,Shot of Glory,country,Its finally payday\nMeeting the boys at my pla...,"[-0.011555955745279789, 0.0511910654604435, 0...."


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def recommend_movies(query_song_embedding, movies_df, top_k=5):
    # Convert movie embeddings to an array
    movie_embeddings = np.vstack(movies_df['embedding'].values)

    # Compute cosine similarity between the query song and all movie embeddings
    similarities = cosine_similarity([query_song_embedding], movie_embeddings)[0]

    # Get the indices of the top_k most similar movies
    top_indices = np.argsort(similarities)[::-1][:top_k]

    # Return the recommended movies with their similarity scores
    recommended_movies = movies_df.iloc[top_indices].copy()
    recommended_movies['similarity'] = similarities[top_indices]

    return recommended_movies


In [ ]:
import random

# Randomly select 5 song indices
random_song_indices = random.sample(range(len(songs_df)), 5)
radom_song_embeddings = songs_df.iloc[random_song_indices]['embedding'].values
recommended_movies_embeddings = []
# Iterate through the randomly selected songs and recommend movies
for index in random_song_indices:
  random_song = songs_df.iloc[index]
  song_embedding = random_song['embedding']
  recommendations = recommend_movies(song_embedding, movies_df)
  recommended_movies_embeddings.append(recommendations['embedding'].values[0])
  print(f"Recommendations for song '{random_song['title']}'':")
  print(recommendations[['title', 'similarity']])
  print("-" * 30)


Recommendations for song 'Don Carlos Act 2 Scene 1'':
                        title  similarity
374925      The King of Spain    0.674567
485749       The Two Brothers    0.614773
193069                 Lovers    0.606475
62967             The Brigand    0.606442
88044   El amor de los amores    0.605546
------------------------------
Recommendations for song 'Internal Permafrost'':
                                       title  similarity
236148  Everyone Sees Noon At Their Own Door    0.604315
185616                          Tainted Love    0.596831
475570                            Surrender!    0.572419
523195                              Addicted    0.569674
212267              A World Without a Mirror    0.559047
------------------------------
Recommendations for song 'Love Is a Lonesome River'':
                  title  similarity
521535   Distant Lights    0.651937
185616     Tainted Love    0.637858
420395        São Jorge    0.635367
102409  Unrequited Love    0.633687
36083  

#GENIUS API CONNECTION - WIP

In [ ]:
#access token - need to remove before publishing
client_access_token = "oHyeePjpeKSzOGxJRFnHFHayfxzGi8CdcD4BvY1nrlQN_Ra1mE1KYGhEgHlcwibT"

In [ ]:
#generate url
search_term = "Missy Elliott"
genius_search_url = f"http://api.genius.com/search?q={search_term}&access_token={client_access_token}"
print(genius_search_url)

http://api.genius.com/search?q=Missy Elliott&access_token=oHyeePjpeKSzOGxJRFnHFHayfxzGi8CdcD4BvY1nrlQN_Ra1mE1KYGhEgHlcwibT


In [ ]:
#get json data from response
import requests
response = requests.get(genius_search_url)
json_data = response.json()
#json_data

In [ ]:
for song in json_data['response']['hits']:
    print(song['result']['full_title'])

Work It by Missy Elliott
WTF (Where They From) by Missy Elliott (Ft. Pharrell Williams)
Get Ur Freak On by Missy Elliott
Lose Control by Missy Elliott (Ft. Ciara & Fatman Scoop)
Gossip Folks by Missy Elliott (Ft. Ludacris)
The Rain (Supa Dupa Fly) by Missy Elliott
I'm Better by Missy Elliott (Ft. Cainon Lamb)
One Minute Man (Video Remix) by Missy Elliott (Ft. Ludacris & Trina)
Hot Boyz (Remix) by Missy Elliott (Ft. Eve, Lil' Mo, Nas & Q-Tip)
Bomb Intro / Pass That Dutch by Missy Elliott


In [ ]:
#get top hit
top_result = json_data['response']['hits'][0]['result']
print(top_result['title'])

Work It


In [ ]:
#find the song in our database
songs_df[songs_df['title'] == top_result['title']]

,title,tag,lyrics,embedding
31819,Work It,country,Spray a little perfume on a thank you card\nFo...,"[0.03182158246636391, 0.09050212055444717, 0.0..."
286409,Work It,rap,Let me see you work it baby\nGon make it twirl...,"[-0.03375029191374779, 0.08421006053686142, 0...."
335553,Work It,rap,I keep my wrists ice cold with the shiniest st...,"[0.0140311848372221, 0.13972307741641998, 0.01..."
419848,Work It,rb,Oh you looking like the right type\nYou know I...,"[-0.0019913041032850742, 0.08658072352409363, ..."


In [ ]:
import random

random_song_indices = random.sample(range(len(songs_df)), 1)
radom_song_embeddings = songs_df.iloc[random_song_indices]['embedding'].values
recommended_movies_embeddings = []
# Iterate through the randomly selected songs and recommend movies
for index in random_song_indices:
  random_song = songs_df.iloc[index]
  print(random_song)
  song_embedding = random_song['embedding']





title                                              Love Letter
tag                                                         rb
lyrics       Shawty you can be my world\nIll buy you diamon...
embedding    [0.025977542623877525, 0.03932156786322594, 0....
Name: 353071, dtype: object


In [2]:
!pip install import_ipynb
import import_ipynb
import songs_embedding

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.8 MB/s eta 0:00:00


ModuleNotFoundError: No module named 'songs_embedding'